In [20]:
import pandas as pd
import numpy as np

In [19]:
tot = 6
p = 3
f = 3

p_studied = ((3/6 * 2/3) +  (3/6 * 1/3))
print('p_studied:', p_studied)
p_pass = 3/6
p_studied_pass = 2/3 # studied given pass
p_studied_fail = 1/3
p_pass_studies = (p_studied_pass * p_studied) / p_studied
print('p_pass_studies:', p_pass_studies)

p_pass_studies = (p_studied_fail * p_studied) / p_studied
print('p_fail_studies:', p_pass_studies)

p_studied: 0.5
p_pass_studies: 0.6666666666666666
p_fail_studies: 0.3333333333333333


In [27]:
np.sqrt(16)

np.float64(4.0)

In [43]:
import math


def gaussian_naive_bayes(x, mu, sigmoid):
    """
    Gaussian Naive Bayes Classifier function f(x)

    Args:
        x (float): Input feature value
        mu (float): Mean of the Gaussian distribution
        sigmoid (float): standard deviation of the Gaussian distribution

    Returns:
        float: Probability density of x under the Gaussian distribution
    """
    exponent = math.exp(-((x - mu) ** 2) / (2 * sigmoid**2))
    return (1 / (math.sqrt(2 * math.pi) * sigmoid)) * exponent

# Example usage
x = 3 #? leaf length
variance = 0.1416 # sigmoid**2 is sigmoid
sigmoid = np.sqrt(variance)
mu = 1.52 # mean
gaus_distribution_cat0 = gaussian_naive_bayes(x, mu, sigmoid)
print("Gaussian Distribution Category 0:", gaus_distribution_cat0)

variance = 0.0776 # sigmoid**2 is sigmoid
mu = 3.88 # mean
sigmoid = np.sqrt(variance)
gaus_distribution_cat1 = gaussian_naive_bayes(x, mu, sigmoid)
print("Gaussian Distribution Category 1:", gaus_distribution_cat1)

0.0342*0.4129*0.5

Gaussian Distribution Category 0: 0.00046381385482350016
Gaussian Distribution Category 1: 0.009749538494816147


0.00706059

In [2]:
Outcome = [
    ["Sunny", "Hot", "High", "Weak", "no"],
    ["Sunny", "Hot", "High", "Strong", "no"],
    ["Overcast", "Hot", "High", "Weak", "yes"],
    ["Rain", "Mild", "High", "Weak", "yes"],
    ["Rain", "Cool", "Normal", "Weak", "yes"],
    ["Rain", "Cool", "Normal", "Strong", "no"],
    ["Overcast", "Cool", "Normal", "Strong", "yes"],
    ["Overcast", "Mild", "High", "Weak", "no"],
    ["Sunny", "Cool", "Normal", "Weak", "yes"],
    ["Rain", "Mild", "Normal", "Weak", "yes"],
]
Test = ["Sunny", "Cool", "High", "Strong"],

In [3]:
def create_training_data(data):
    return np.array(data, dtype=np.str_)

train_data = create_training_data(Outcome)
train_data

array([['Sunny', 'Hot', 'High', 'Weak', 'no'],
       ['Sunny', 'Hot', 'High', 'Strong', 'no'],
       ['Overcast', 'Hot', 'High', 'Weak', 'yes'],
       ['Rain', 'Mild', 'High', 'Weak', 'yes'],
       ['Rain', 'Cool', 'Normal', 'Weak', 'yes'],
       ['Rain', 'Cool', 'Normal', 'Strong', 'no'],
       ['Overcast', 'Cool', 'Normal', 'Strong', 'yes'],
       ['Overcast', 'Mild', 'High', 'Weak', 'no'],
       ['Sunny', 'Cool', 'Normal', 'Weak', 'yes'],
       ['Rain', 'Mild', 'Normal', 'Weak', 'yes']], dtype='<U8')

In [4]:
class_data = np.unique(train_data[:, -1], return_counts=True) # ['no', "yes"]
class_count = class_data[1]
class_names = class_data[0]

print(class_count)
print(class_data)


[4 6]
(array(['no', 'yes'], dtype='<U8'), array([4, 6]))


In [5]:
element_counts = {}
for row in train_data:
    for element in row:
        # print(element_counts.get(element, 0))
        element_counts[element] = element_counts.get(element, 0) + 1

print(element_counts)

{np.str_('Sunny'): 3, np.str_('Hot'): 3, np.str_('High'): 5, np.str_('Weak'): 7, np.str_('no'): 4, np.str_('Strong'): 3, np.str_('Overcast'): 3, np.str_('yes'): 6, np.str_('Rain'): 4, np.str_('Mild'): 3, np.str_('Cool'): 4, np.str_('Normal'): 5}


In [6]:
def compute_prior_probabilities(train_data):
    """
    Calculate prior probability P(Play Tennis = Yes/No)

    Args:
        train_data: Training dataset

    Returns:
        Array of prior probabilities [P(No), P(Yes)]
    """

    total_samples = len(train_data)

    #? ( array(['no', 'yes'], dtype='<U8'), array([4, 6]) )
    class_data = np.unique(train_data[:, -1], return_counts=True)
    class_count = class_data[1]
    class_names = class_data[0]

    prior_probabilities = class_count / total_samples

    return prior_probabilities, class_names

prior_probs, _ = compute_prior_probabilities(train_data)
print("P('Play Tennis') = No is", prior_probs[0]) # prob of not playing
print("P('Play Tennis') = Yes is", prior_probs[1]) # prob of yes playing

P('Play Tennis') = No is 0.4
P('Play Tennis') = Yes is 0.6


In [7]:
total_feature = train_data.shape
total_feature

(10, 5)

In [8]:
def calculate_conditional_probabilities(feature_names, prior_probs, train_data):
    """
        Compute probality of given features
        Args:
            feature_names (array[str]): array of feature name ['Sunny', 'Cool', 'High', 'Strong']
            prior_prob (array[float]): probability of each class

        Returns:
            Array of conditional probability for [P(features|no), P(features|yes)]
    """

    count_features = {} # number of each feature exist
    total_sample, total_feature = train_data.shape[0], train_data.shape[1]

    #? calc prior prob of given features
    for i in range(0, total_feature):
        columns = train_data[:, i] #? extract entire column for each feature
        # print(columns)
        for sample in columns:
            #? count feature's sample
            # if found features but feature isn't added, add 1 to current feature count, else current feature count set 1
            if sample in feature_names:
                count_features[sample] = count_features.get(sample, 0) + 1

    #? {} represent set, doesn't allow duplicate value
    #? [] mean array, allow duplicate value
    feature_probs = [count_features/total_feature for count_features in count_features.values()]
    #? P_Outlook, P_Temperature, P_Humidity, P_Wind

    return feature_probs

feature_probs = calculate_conditional_probabilities(['Sunny', 'Cool', 'High', 'Strong'], prior_probs, train_data)
feature_probs

[0.6, 0.8, 1.0, 0.6]

In [ ]:
def predict_tennis(feature_probs, prior_probs):
    #? calc product of feature_probs for each prior_probs
    """_summary_

    Args:
        feature_probs (ndarray[float]): prob of each features
        prior_probs (ndarray[float]): class prob

    Return:
        prior probabilities [P(No), P(Yes)]
    """

    P_yes = np.dot(feature_probs, prior_probs[0])
    P_no = np.dot(feature_probs, prior_probs[1])

    # pred_class_idx = np.arg

    # print(np.prod(P_yes), np.prod(P_no))
    return np.prod(P_yes), np.prod(P_no)

tennis = predict_tennis(feature_probs, prior_probs)
tennis

AttributeError: module 'numpy' has no attribute 'arg'